In [1]:
import torch
import torchvision


class BasicModel(torch.nn.Module):
    """
    This is a basic backbone for SSD.
    The feature extractor outputs a list of 6 feature maps, with the sizes:
    [shape(-1, output_channels[0], 38, 38),
     shape(-1, output_channels[1], 19, 19),
     shape(-1, output_channels[2], 10, 10),
     shape(-1, output_channels[3], 5, 5),
     shape(-1, output_channels[3], 3, 3),
     shape(-1, output_channels[4], 1, 1)]
     where "output_channels" is the same as cfg.BACKBONE.OUT_CHANNELS
    """
    def __init__(self, cfg):
        super().__init__()
        #print('hello!, here is the def_init_')
        image_size = cfg.INPUT.IMAGE_SIZE
        #print('image_size=',image_size)
        output_channels = cfg.MODEL.BACKBONE.OUT_CHANNELS
        #print('output_channel=',output_channels)
        self.output_channels = output_channels
        image_channels = cfg.MODEL.BACKBONE.INPUT_CHANNELS
        #print('image_channels=',image_channels)
        self.output_feature_size = cfg.MODEL.PRIORS.FEATURE_MAPS
        #print('self.output_feature_size=',self.output_feature_size)
    #Define the structure
    #1.Classical VGG
        base=[]
        base.append(torch.nn.Conv2d(in_channels=image_channels, out_channels=64,kernel_size=3,stride=1,padding=1))
        base.append(torch.nn.MaxPool2d(kernel_size=2, stride=2))
        base.append(torch.nn.ReLU())
        base.append(torch.nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3,stride=1,padding=1))
        base.append(torch.nn.MaxPool2d(kernel_size=2, stride=2))
        base.append(torch.nn.ReLU())
        base.append(torch.nn.Conv2d(in_channels=128, out_channels=128,kernel_size=3,stride=1,padding=1)) 
        base.append(torch.nn.ReLU())
        base.append(torch.nn.Conv2d(in_channels=128, out_channels=256,kernel_size=3,stride=2,padding=1)) #output[0]
        base.append(torch.nn.ReLU())
        self.base = torch.nn.Sequential(*base)
    #2. extra_layers (ReLU will be used in the foward function)
        extra_layer=[]
        extra_layer.append(torch.nn.Conv2d(in_channels=256, out_channels=256,kernel_size=3,stride=1,padding=1))
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=256, out_channels=512,kernel_size=3,stride=2,padding=1)) #ouput[1]
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=512, out_channels=512,kernel_size=3,stride=1,padding=1))
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=512, out_channels=256,kernel_size=3,stride=2,padding=1)) #output[2]
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=256, out_channels=256,kernel_size=3,stride=1,padding=1))
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=256, out_channels=256,kernel_size=3,stride=2,padding=1)) #output[3]
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=256, out_channels=256,kernel_size=3,stride=1,padding=1))
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=256, out_channels=128,kernel_size=3,stride=2,padding=1))  #output[4]
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=128, out_channels=256,kernel_size=3,stride=1,padding=1))
        #need ReLU
        extra_layer.append(torch.nn.Conv2d(in_channels=256, out_channels=128,kernel_size=3,stride=2,padding=0))  #output[5]
        #need ReLU
        self.extra_layer=torch.nn.Sequential(*extra_layer)
        
    def forward(self, x):
        """
        The forward functiom should output features with shape:
            [shape(-1, output_channels[0], 38, 38),
            shape(-1, output_channels[1], 19, 19),
            shape(-1, output_channels[2], 10, 10),
            shape(-1, output_channels[3], 5, 5),
            shape(-1, output_channels[3], 3, 3),
            shape(-1, output_channels[4], 1, 1)]
        We have added assertion tests to check this, iteration through out_features,
        where out_features[0] should have the shape:
            shape(-1, output_channels[0], 38, 38),
        """
        #print('hello, this is the forward')
        out_features = []
        #The output from the base,i.e. output[0]
        x=self.base(x)
        out_features.append(x)
        #For other outputs:
        for i, f in enumerate(self.extra_layer): #i means the index and f means the function of the layer
            if i== len(self.extra_layer):
                x= f(x)
            else:
                x= torch.nn.functional.relu(f(x),inplace=True)
            if i % 2 == 1:
                out_features.append(x)
                #print('layer:',f)
        #print('out_features',out_features[0].shape)
            
#         for idx, feature in enumerate(out_features):
#             out_channel = self.output_channels[idx]
#             feature_map_size=self.output_feature_size[idx]
#             #print('out_channel',feature.shape[1:])
#             expected_shape = (out_channel, feature_map_size, feature_map_size)
#             assert feature.shape[1:] == expected_shape, \
#                 f"Expected shape: {expected_shape}, got: {feature.shape[1:]} at output IDX: {idx}"
        return tuple(out_features)



In [2]:
vgg_16=torchvision.models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/xyzhu/.cache/torch/checkpoints/vgg16-397923af.pth


In [3]:
print(vgg_16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
test=BasicModel()

TypeError: __init__() missing 1 required positional argument: 'cfg'

In [1]:
from torch import nn
from ssd.modeling.backbone.vgg import VGG
from ssd.modeling.backbone.basic import BasicModel
from ssd.modeling.box_head.box_head import SSDBoxHead
from ssd.utils.model_zoo import load_state_dict_from_url
from ssd.modeling.backbone.resnet_base import ResNetBase
from ssd import torch_utils

class SSDDetector(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.backbone = build_backbone(cfg)
        self.box_head = SSDBoxHead(cfg)
        print(
            "Detector initialized. Total Number of params: ",
            f"{torch_utils.format_params(self)}")
        print(
            f"Backbone number of parameters: {torch_utils.format_params(self.backbone)}")
        print(
            f"SSD Head number of parameters: {torch_utils.format_params(self.box_head)}")

    def forward(self, images, targets=None):
        features = self.backbone(images)

        detections, detector_losses = self.box_head(features, targets)
        if self.training:
            return detector_losses
        return detections


def build_backbone(cfg):
    backbone_name = cfg.MODEL.BACKBONE.NAME
    print(backbone_name)
    if backbone_name == "basic":
        model = BasicModel(cfg)
        return model
    if backbone_name == "vgg":
        model = VGG(cfg)
        if cfg.MODEL.BACKBONE.PRETRAINED:
            state_dict = load_state_dict_from_url(
                "https://s3.amazonaws.com/amdegroot-models/vgg16_reducedfc.pth")
            model.init_from_pretrain(state_dict)
    if backbone_name == "resnet":
        model_urls = {
            'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
            'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
            'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
            'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
            'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',}
        name_dict = {18: 'resnet18', 34: 'resnet34', 50: 'resnet50', 101: 'resnet101', 152: 'resnet152'}
        layers_dict = {18: [2, 2, 2, 2], 34: [3, 4, 6, 3], 50: [3, 4, 6, 3], 
                       101: [3, 4, 23, 3], 152: [3, 8, 36, 3]}
        block_dict = {18: BasicBlock, 34: BasicBlock, 50: Bottleneck, 101: Bottleneck, 152: Bottleneck}
        model = resnet_base(cfg)
        depth = cfg.MODEL.BACKBONE.DEPTH
        model = ResNetBase(block_dict[depth], layers_dict[depth],1, **kwargs)
        if cfg.MODEL.BACKBONE.PRETRAINED:
            state_dict = load_state_dict(model_zoo.load_url(model_urls[name_dict[depth]]))
            model.init_from_pretrain(state_dict)
        return model


ModuleNotFoundError: No module named 'ssd'